## 9. Estructura de paquetes, módulos y ficheros.

### Ficheros y entrada/salida, apertura de ficheros e introducción de contextos.

Para abrir un fichero usamos la función `open()`, que tiene como parámetros el nombre y ruta del fichero y el modo de apertura, una serie de caracteres que definen como se va a usar el fichero abierto.

```python
>>> f = open('workfile', 'w')
```

- `r` para solo lectura
- `w` para solo escritura, siendo borrado cualquier fichero que ya existiera
- `a` para añadir contenido al final del fichero
- `r+` para lectura y escritura

Por defecto, si se omite el modo, se asume que es `r`. 

Además, también por defecto, un fichero se abre en modo texto, lo que significa que lee el contenido como cadenas de texto, aplicando transformaciones que dependen de la plataforma, como es la conversión de los salos de líneas, entre los formatos de Unix o de Windows, aplicando esta transformación tanto a la lectura como a la escritura.


Para abrir en modo **binario** añadimos `b` al final de la cadena de texto que indica el modo.

La función `open` devuelve un descriptor que apunta al fichero, no el contenido. Una vez se deja de usar se tiene que llamar al método `f.close()` del descriptor para cerrar el fichero.


Una buena práctica, para evitar la necesidad de cerrar el descriptor, es usar la palabra reservada `with`. Esto crea lo que se define en Python como un **contexto**.

```python
>>> with open('workfile') as f:
...     read_data = f.read()
>>> f.closed
True
```

En este ejemplo, la variable `f` solo existe en el bloque que se ejecuta dentro del `with`, y se llama a `f.close()` una vez que la ejecución sale del bloque.

#### Lectura de contenido

##### Método `read(size)`

Lee la cantidad de información del fichero indicada en el parámetro size. Si se omite, lee todo el contenido del fichero.

##### Método `readline()`

Cada vez que se llama, lee la siguiente linea del fichero, es decir, hasta el siguiente salto de línea.

Para iterar sobre las líneas de un fichero, es más eficiente hacerlo directamente sobre el descriptor del fichero.

```python
>>> for line in f:
...     print(line, end='')
...
This is the first line of the file.
Second line of the file
```

Puedes leer todas las lineas de un fichero y pasarlas a una lista con `list(f)` o `f.readlines()`.

#### Escritura de contenido

##### Método `write(string)`

El método `write(string)` escribe el contenido de string en el fichero, devolviendo el número de caracteres escritos.

#### Moverse por el fichero

El método `f.tell()` devuelve un entero con la posición en la que nos encontramos en el fichero, representado por el número de bytes desde el comienzo, cuando se abre en modo binario.

Con el método `f.seek(offset, from_what)` nos podemos desplazar por el fichero. La posición a la que nos movemos se calcula añadiendo lo indicado en el offset a la posición refernciada en from_what, que es 0 para el comienzo del fichero, 1 para la posición actual del descriptor y 2 para el final del fichero.


### Paquetes y módulos

Cuando salimos del interprete y volvemos a entrar, todas las definiciones y declaraciones que hemos realizado se pierden. Para escribir cualquier programa que vaya más allá de lo que el uso puntual del interprete requiera, debemos escribirlo en un fichero de texto con la extensión `.py`.

El interprete de Python tratara cada uno de estos ficheros como un módulo, `module`. Cada definición de nombres, variables, funciones, etc. que contenga un módulo puede ser importado a otro módulo y ser usado en este.

Por ejemplo, si tenemos el siguiente fichero, `fibo.py`:

```python
# Módulo números de Fibonacci


def fib(n):
    a, b = 0, 1
    while b < n:
        print(b, end=' ')
        a, b = b, a+b
    print()

        
def fib2(n):
    result = []
    a, b = 0, 1
    while b < n:
        result.append(b)
        a, b = b, a+b
    return result
```

- Podemos importar estas funciones en el intérprete o en cualquier otro módulo.
- Cada módulo define una variable global `__name__`, que contiene el nombre del módulo.

```python
>>> import fibo
>>> fibo.fib(1000)
1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987
>>> fibo.fib2(100)
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
>>> fibo.__name__
'fibo'
```

Un módulo puede tener instrucciones además de definiciones. Estas instrucciones suelen tener como objetivo inicializar el módulo, y **sólo se ejecutarán la primera vez que se hace un `import` del módulo**. También se ejecutan cuando el fichero de módulo se ejecuta como un script.

Todo módulo tiene su propia tabla de símbolos, por lo que cualquiera puede usar variables globales sin preocuparse de que estas colisionen entre módulos.


Se pueden importar nombres desde un módulo directamente a la tabla de nombres de otro módulo.

```python
>>> from fibo import fib, fib2
>>> fib(500)
1 1 2 3 5 8 13 21 34 55 89 144 233 377
```

Y también se pueden importar todos los nombres que define un módulo.

```python
>>> from fibo import *
>>> fib(500)
1 1 2 3 5 8 13 21 34 55 89 144 233 377
```

Esto importa todos los nombres excepto los que empiezan por `_`.

### Ejecutar módulos como scripts

Puedes ejecutar cualquier módulo usando el comando de `python`.

```
$ python fibo.py
```

Si el código del módulo se ejecuta de esta forma, entonces la variable `__name__` del módulo toma el valor de `__main__`, lo que significa que podemos diferenciar que parte del modulo se ejecutará cuando se importa y cuando se ejecuta como script.

```python
if __name__ == "__main__":
    # Sólo ejecuta esto cuando es un script
    pass 
```

### Paquetes

Los paquetes o *packages* de Python son una forma de estructurar los módulos de Python. 

Se considera un package cualquier carpeta que tenga un fichero llamado `__init__.py`. Este fichero puede estar vacío, pero también puede incluir código para inicializar el paquete. Un paquete puede a su vez contener, o no, uno o varios módulos.

```
sound/                          
      __init__.py               
      formats/                  
              __init__.py
              wavread.py
              wavwrite.py
              aiffread.py
              aiffwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  
              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...
```

Si se intenta hacer un `from sound.effects import *` cabría esperar que esto importara todos los nombres definidos en todos los módulos de dentro del paquete y sus sub-paquetes, pero sería muy costoso, por que se sigue la convención de que el autor de un paquete declare la variable `__all__` como una lista de nombres de los módulos que se importarán cuando se use el * para importar todo.